# Notes Part 2

Overfitting - does *not* mean that you're training loss is lower than validation loss! 

A well fit will almost always have training loss lower than the validation loss.

Overfitting means - you're actually seeing your validation loss gettting worse.

### Five steps to avoid overfitting

More data

Data augmentation 

Generalizable architectures

Regularization

Reduce architecture complexity

### Steps to a basic modern CNN model

Matmul

Relu / Init

FC Forward

FC backward

Train loop

Conv

Optim

Batch-Norm

ResNet

If you want to develop your own library in Jupyter notebooks, follow notebook2script.py

# Matrix Multiplication

In [2]:
from fastai.basics import *
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math
import matplotlib as mpl
import torch
import matplotlib.pyplot as plt
from torch import tensor

In [3]:
import time

def timeit(func):
    def wrapper(*args, **kwargs):
        now = time.time()
        retval = func(*args, **kwargs)
        print('{} took {:.5f}s'.format(func.__name__, time.time() - now))
        return retval
    return wrapper

In [4]:
# python only matrix multiplication

def matmul(a, b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac):
                c[i, j] += a[i,k] * b[k,j]
                
    return c

In [5]:
m1 = torch.rand((5, 784))
m2 = torch.rand((784, 10))

In [6]:
m1.shape, m2.shape

(torch.Size([5, 784]), torch.Size([784, 10]))

In [7]:
%timeit t = matmul(m1, m2)

922 ms ± 3.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
t = matmul(m1, m2)

In [9]:
t.shape

torch.Size([5, 10])

About 1 second per matrix multiply, so if you have a 50,000 long dataset (MNIST), will take you 50,000 seconds 

### Every layer would take about 10 hours!

# How do we speed things up? Write in something other than python!

### PyTorch behind the scenes is using a library called a10

Elementwise operations

Operators +, -, *, >, <, == are usually element-wise

In [10]:
# Example

a = tensor([10., 6, -4])
b = tensor([2., 8, 7])
a,b 

(tensor([10.,  6., -4.]), tensor([2., 8., 7.]))

In [11]:
a + b

tensor([12., 14.,  3.])

In [12]:
(a < b).float().mean()

tensor(0.6667)

In [13]:
m = tensor([[1.,2,3], [4,5,6], [7,7,8]]); m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 7., 8.]])

## Frobenius norm:

$$\| A \|_F = \left( \sum_{i,j=1}^n | a_{ij} |^2 \right)^{1/2}$$

hint - you can copy LaTeX from wikipedia 

All it is:
**Matrix times itself, .sum(), .sqrt()**

In [14]:
(m*m).sum().sqrt()

tensor(15.9060)

In [19]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac == br
    
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            # TAKING OUT THE THIRD LOOP
            # REPLACING with colon, meaning entirety of that axis
            # i is the row number, j is the column
            # [i:, ] all of row i
            # [:,j] all of column j
            c[i, j] = (a[i,:] * b[:,j]).sum()
            
    return c

# this is running in C, not Python!

In [20]:
%timeit -n 10 _=matmul(m1, m2)

1.55 ms ± 76.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%timeit t =matmul(m1, m2)

1.35 ms ± 62.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Now have matrix multiplication around 63 microseconds

# Broadcasting

APL library

Use implicit broadcasted loops instead of for loops

In [22]:
a

tensor([10.,  6., -4.])

In [23]:
a > 0

tensor([ True,  True, False])

Element wise broadcasting
Broadcast a scalar to a tensor

The term **broadcasting** describes how arrays with different shapes are treated during arithmetic operations.  The term broadcasting was first used by Numpy.

From the [Numpy Documentation](https://docs.scipy.org/doc/numpy-1.10.0/user/basics.broadcasting.html):

    The term broadcasting describes how numpy treats arrays with 
    different shapes during arithmetic operations. Subject to certain 
    constraints, the smaller array is “broadcast” across the larger 
    array so that they have compatible shapes. Broadcasting provides a 
    means of vectorizing array operations so that looping occurs in C
    instead of Python. It does this without making needless copies of 
    data and usually leads to efficient algorithm implementations.
    
In addition to the efficiency of broadcasting, it allows us to write less code, which typically leads to fewer errors.

### Broadcasting acts like a for loop, and allows us to write at C speed

Broadcast one row accross multiple rows of a tensor super fast

In [24]:
a + 1

tensor([11.,  7., -3.])

In [25]:
m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 7., 8.]])

In [26]:
m*2

tensor([[ 2.,  4.,  6.],
        [ 8., 10., 12.],
        [14., 14., 16.]])

In [28]:
c = tensor([10.,20,30]); c

tensor([10., 20., 30.])

In [29]:
t = c.expand_as(m)

In [30]:
t

tensor([[10., 20., 30.],
        [10., 20., 30.],
        [10., 20., 30.]])

In [31]:
t.storage()

 10.0
 20.0
 30.0
[torch.FloatStorage of size 3]

In [32]:
t.stride(), t.shape

((0, 1), torch.Size([3, 3]))

In [36]:
c # rank 1 tensor

tensor([10., 20., 30.])

In [37]:
c.unsqueeze(0) # rank 2 tensor

tensor([[10., 20., 30.]])

In [38]:
c.unsqueeze(1) # rank 2 tensor that looks like a column

tensor([[10.],
        [20.],
        [30.]])

In [39]:
c.shape, c.unsqueeze(0).shape, c.unsqueeze(1).shape

(torch.Size([3]), torch.Size([1, 3]), torch.Size([3, 1]))

In [41]:
c[None,:].shape, c[:None].shape

(torch.Size([1, 3]), torch.Size([3]))

In [43]:
c[None,:] # same as c.unsqueeze(0) - says please an axis in this row

tensor([[10., 20., 30.]])

In [45]:
c[:,None] # same as c.unsqueeze(1) - says please an axis in this column

tensor([[10.],
        [20.],
        [30.]])

In [46]:
c[:,None].expand_as(m) # Broadcast as columns

tensor([[10., 10., 10.],
        [20., 20., 20.],
        [30., 30., 30.]])

In [49]:
m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 7., 8.]])

In [47]:
m + c[:, None]

tensor([[11., 12., 13.],
        [24., 25., 26.],
        [37., 37., 38.]])

In [48]:
m + c[None,:]

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 27., 38.]])

Matrix multiplication with Broadcasting

In [50]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
#       c[i,j] = (a[i,:]          * b[:,j]).sum() # previous
        c[i]   = (a[i  ].unsqueeze(-1) * b).sum(dim=0) 
# unsqueeze(-1) or a[i, None]
# broadcasting - down 254 microseconds
    return c

In [51]:
# can replace every ;, in c[;, None] with a single . 
# this is convenient when you want to work with really high rank tensors

In [52]:
%timeit t = matmul(m1, m2)

297 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Broadcasting Rules (same in numpy, pytorch and tensorflow)

When operating on two arrays/tensor, Numpy /Pytorch compares their shapes **element-wise**.

Starts with **trailing dimensions**, and works its way forward. Two dimensions are compatible when ...

- They are equal, or

- One of them is 1, in which case that dimension is broadcasted to make it the same size

Arrays do not need to have he same number of dimensions. 

For example, if you have a `256 * 256 * 3` RGB array, and you want to scale all colors of the image, you can multiply the entire image by a one-dim array with 3 values

Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:
    
``` Python
    Img: 256 x 256 x 3 (3d array)
    Scale: 3 (1d array)
    Result: 256 x 256 x 3 (3d array) # same as input!!!
```

# Einstein Summation

Einstein summation (`einsum`) is a compact representation for combining products and sums in a general way. From the numpy docs:

"The subscripts string is a comma-separated list of subscript labels, where each label refers to a dimension of the corresponding operand. Whenever a label is repeated it is summed, so `np.einsum('i,i', a, b)` is equivalent to `np.inner(a,b)`. If a label appears only once, it is not summed, so `np.einsum('i', a)` produces a view of a with no changes."

In [61]:
def matmul(a,b): return torch.einsum('ik,kj -> ij', a, b)

In [57]:
# c[i,j] += a[i,k] * b[k,j]

#   ik, kj -> ij

# c[i,j] = (a[i,:] * b[:,j]).sum()

input has i rows, output has i rows
input has j columns, output has j columns

k is what we're mutliplying the rows and the columns by

In [58]:
# any place where something is repeated, and take the dot product over them
# 'ik, kj -> ij'

In [59]:
%timeit t = matmul(m1, m2)

75.6 µs ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
# say we needed batchwise matrix multiplication??

def matmul(a,b): return torch.einsum('bik,bkj -> bij', a, b)

# Pytorch Operation

use pytorch's function or operator directly for matrix multiplication!

In [62]:
%timeit t = m1.matmul(m2) 

12.2 µs ± 5.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


From regular Python to C in Pytorch, we went from 922 ms to 12.2 µs!!!